In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
import time

import requests
import xmltodict

# 1. 초기 설정
base_url = "http://apis.data.go.kr/1613000/RTMSDataSvcOffiRent/getRTMSDataSvcOffiRent"
service_key = "8afa4c269ca52320f0ba72cd7d98b8d8003a8c88cafcb41d001dddbca8debf89"

# 수집 대상 설정
lawd_codes = {
    "서울(종로)": "11110",
    "충남(아산)": "44210",
    "전남(무안)": "46830",
    "전북(완주)": "45710",
    "충남(홍성)": "44800",
    "경북(안동)": "47170",
}

# 2023년 1월부터 12월까지 문자열 생성
months = [f"2025{str(m).zfill(2)}" for m in range(6, 12)]

all_data = []

# 2. 중첩 루프 실행
for deal_ymd in months:
    print(f"{deal_ymd} 데이터 수집 시작...")

    for code in lawd_codes:
        params = {
            "serviceKey": requests.utils.unquote(service_key),
            "LAWD_CD": code,
            "DEAL_YMD": deal_ymd,
        }

        try:
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                data_dict = xmltodict.parse(response.text)

                # 데이터 계층 구조 진입 및 결측치 방어
                body = data_dict.get("response", {}).get("body", {})
                items = body.get("items") if body else None

                if items and "item" in items:
                    item_list = items["item"]

                    # 단건(dict)인 경우 리스트로 강제 변환
                    if isinstance(item_list, dict):
                        item_list = [item_list]

                    all_data.extend(item_list)
                    print(f"  [{deal_ymd}] 지역 {code}: {len(item_list)}건 추가")
                else:
                    # 데이터가 없는 경우 (None이거나 items가 비어있음)
                    print(f"  [{deal_ymd}] 지역 {code}: 데이터 없음")
            else:
                print(f"  서버 응답 에러: {response.status_code}")

            # API 서버 부하 방지 (초당 호출 제한 준수)
            time.sleep(0.2)

        except Exception as e:
            print(f"  [{deal_ymd}] 지역 {code} 오류: {e}")

# 3. 결과 요약
print("-" * 30)
print(f"수집 종료: 총 {len(all_data)}건의 데이터가 리스트에 저장되었습니다.")

202506 데이터 수집 시작...
  [202506] 지역 서울(종로): 데이터 없음
  [202506] 지역 충남(아산): 데이터 없음
  [202506] 지역 전남(무안): 데이터 없음
  [202506] 지역 전북(완주): 데이터 없음
  [202506] 지역 충남(홍성): 데이터 없음
  [202506] 지역 경북(안동): 데이터 없음
202507 데이터 수집 시작...
  [202507] 지역 서울(종로): 데이터 없음
  [202507] 지역 충남(아산): 데이터 없음
  [202507] 지역 전남(무안): 데이터 없음
  [202507] 지역 전북(완주): 데이터 없음
  [202507] 지역 충남(홍성): 데이터 없음
  [202507] 지역 경북(안동): 데이터 없음
202508 데이터 수집 시작...
  [202508] 지역 서울(종로): 데이터 없음
  [202508] 지역 충남(아산): 데이터 없음
  [202508] 지역 전남(무안): 데이터 없음
  [202508] 지역 전북(완주): 데이터 없음
  [202508] 지역 충남(홍성): 데이터 없음
  [202508] 지역 경북(안동): 데이터 없음
202509 데이터 수집 시작...
  [202509] 지역 서울(종로): 데이터 없음
  [202509] 지역 충남(아산): 데이터 없음
  [202509] 지역 전남(무안): 데이터 없음
  [202509] 지역 전북(완주): 데이터 없음
  [202509] 지역 충남(홍성): 데이터 없음
  [202509] 지역 경북(안동): 데이터 없음
202510 데이터 수집 시작...
  [202510] 지역 서울(종로): 데이터 없음
  [202510] 지역 충남(아산): 데이터 없음
  [202510] 지역 전남(무안): 데이터 없음
  [202510] 지역 전북(완주): 데이터 없음
  [202510] 지역 충남(홍성): 데이터 없음
  [202510] 지역 경북(안동): 데이터 없음
202511 데이터 수집 시작...
  [202511]

서울 제외 필터링

In [ ]:
import time

import pandas as pd
import requests
import xmltodict

# 1. 지역 설정
target_regions = {
    "서울(종로)": "11110",
    "충남(아산)": "44210",
    "전남(무안)": "46830",
    "전북(완주)": "45710",
    "충남(홍성)": "44800",
    "경북(안동)": "47170",
}

past_ymd = "202412"
present_ymd = "202512"
# 환경변수가 없다면 여기에 직접 인증키 문자열을 넣으세요
service_key = "8afa4c269ca52320f0ba72cd7d98b8d8003a8c88cafcb41d001dddbca8debf89"


def get_real_cost_avg(ymd, code):
    params = {
        "serviceKey": requests.utils.unquote(service_key),
        "LAWD_CD": code,
        "DEAL_YMD": ymd,
    }
    base_url = (
        "http://apis.data.go.kr/1613000/RTMSDataSvcOffiRent/getRTMSDataSvcOffiRent"
    )

    try:
        res = requests.get(base_url, params=params)
        # API 결과가 성공이 아닐 경우 확인
        if res.status_code != 200:
            print(f"  ❌ API 연결 실패 ({res.status_code})")
            return None

        data = xmltodict.parse(res.text)

        # API 서버 에러 메시지 확인 (인증키 오류 등)
        header = data.get("response", {}).get("header", {})
        if header.get("resultCode") != "00":
            print(f"  ❌ API 에러: {header.get('resultMsg')}")
            return None

        body = data.get("response", {}).get("body", {})
        items = body.get("items")
        if not items or "item" not in items:
            return None

        item_list = items["item"]
        if isinstance(item_list, dict):
            item_list = [item_list]

        costs = []
        for it in item_list:
            # [수정] 한글 태그와 영문 태그 모두 대응 (API 버전에 따라 다름)
            dep_str = it.get("보증금액") or it.get("deposit") or "0"
            rent_str = it.get("월세금액") or it.get("monthlyRent") or "0"

            dep = int(dep_str.replace(",", ""))
            rent = int(rent_str.replace(",", ""))

            # 실질 주거비 (5.5% 적용)
            real_cost = rent + (dep * 0.055 / 12)
            costs.append(real_cost)

        return sum(costs) / len(costs) if costs else None

    except Exception as e:
        print(f"  🔥 상세 에러 발생 ({code}, {ymd}): {e}")
        return None


# 실행 로직은 동일...
comparison_list = []
for name, code in target_regions.items():
    print(f"🔍 {name} ({code}) 데이터 분석 중...")
    avg_24 = get_real_cost_avg(past_ymd, code)
    avg_25 = get_real_cost_avg(present_ymd, code)

    if avg_24 is not None and avg_25 is not None:
        diff = avg_25 - avg_24
        rate = (diff / avg_24) * 100
        comparison_list.append(
            {
                "지역명": name,
                "24년12월": int(avg_24),
                "25년12월": int(avg_25),
                "증감액": int(diff),
                "증감률(%)": round(rate, 2),
            }
        )
    else:
        print(f"  ⚠️ {name} 데이터 부족으로 계산 불가")
    time.sleep(0.5)

if comparison_list:
    df_final = pd.DataFrame(comparison_list).sort_values(
        by="증감률(%)", ascending=False
    )
    print("\n" + df_final.to_markdown(index=False))
else:
    print("\n❌ 모든 지역의 데이터를 수집하지 못했습니다.")

🔍 서울(종로) (11110) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 서울(종로) 데이터 부족으로 계산 불가
🔍 충남(아산) (44210) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 충남(아산) 데이터 부족으로 계산 불가
🔍 전남(무안) (46830) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 전남(무안) 데이터 부족으로 계산 불가
🔍 전북(완주) (45710) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 전북(완주) 데이터 부족으로 계산 불가
🔍 충남(홍성) (44800) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 충남(홍성) 데이터 부족으로 계산 불가
🔍 경북(안동) (47170) 데이터 분석 중...
  ❌ API 에러: OK
  ❌ API 에러: OK
  ⚠️ 경북(안동) 데이터 부족으로 계산 불가

❌ 모든 지역의 데이터를 수집하지 못했습니다.


In [19]:
# 1. 서울(11로 시작하는 코드)이 아닌 데이터만 추출
df_non_seoul = df_refined[~df_refined["행정코드"].str.startswith("11")].copy()

print(f"✅ 서울 제외 데이터 수: {len(df_non_seoul)}건")
print(df_non_seoul.head())

✅ 서울 제외 데이터 수: 222건
     행정코드             단지   보증금액  월세금액  건축년도  실질주거비
10  44200        라온포레스트뷰   1000    46  2018     51
11  44200  천안아산역 EG the1   1000    95  2025    100
12  44200      코아루웰메이드시티  11600     0  2020     53
13  44200             1동  13000     0  2022     60
14  44200             1동    300    45  2016     46


지역별 평균 주거비 비교 

In [20]:
# 2. 지역별(행정코드별) 평균 실질주거비 계산
region_summary = (
    df_refined.groupby("행정코드")["실질주거비"]
    .agg(["mean", "count"])
    .sort_values(by="mean", ascending=False)
)

# 가독성을 위해 소수점 정리
region_summary["mean"] = region_summary["mean"].round(0).astype(int)

print("📊 지역별 평균 주거비 및 거래 건수")
print(region_summary)

📊 지역별 평균 주거비 및 거래 건수
       mean  count
행정코드              
11110   103     60
44200    68     60
46840    55     60
47170    41     60
44800    36     42


In [17]:
import pandas as pd

# 1. 데이터프레임 생성
df = pd.DataFrame(all_data)

# 2. 전용면적을 제외한 핵심 컬럼만 매핑
mapping = {
    "sggCd": "행정코드",
    "offiNm": "단지",
    "deposit": "보증금액",
    "monthlyRent": "월세금액",
    "buildYear": "건축년도",
}

# 존재하는 컬럼만 필터링 후 이름 변경
actual_columns = [col for col in mapping.keys() if col in df.columns]
df_refined = df[actual_columns].copy()
df_refined.rename(columns=mapping, inplace=True)

# 3. 데이터 타입 변환 (TypeError 방지의 핵심)
# 보증금과 월세에서 콤마를 제거하고 숫자로 변환합니다.
df_refined["보증금액"] = pd.to_numeric(
    df_refined["보증금액"].astype(str).str.replace(",", ""), errors="coerce"
).fillna(0)
df_refined["월세금액"] = pd.to_numeric(
    df_refined["월세금액"].astype(str).str.replace(",", ""), errors="coerce"
).fillna(0)

# 4. 실질 주거비 계산 (전환율 5.5% 가정)
# 전용면적 없이 '총액' 관점에서만 계산합니다.
df_refined["실질주거비"] = df_refined["월세금액"] + (
    df_refined["보증금액"] * 0.055 / 12
)
df_refined["실질주거비"] = df_refined["실질주거비"].round(0).astype(int)

print("✅ 전용면적 제외, 실질 주거비 산출 완료")
print(df_refined.head())

✅ 전용면적 제외, 실질 주거비 산출 완료
    행정코드             단지   보증금액  월세금액  건축년도  실질주거비
0  11110         이지마루종로   1000   150  2021    155
1  11110           BS타워   1000    75  2019     80
2  11110    경희궁의 아침 4단지   2000   160  2004    169
3  11110      광화문 스페이스본  33000     0  2008    151
4  11110  휴스턴오피스텔 창경궁 C   1000   110  2022    115


In [3]:
data_dict.keys()

dict_keys(['response'])

In [71]:
items = data_dict["response"]["body"]["items"]

if items is not None:
    keys = items.keys()
    # 후속 작업 진행
else:
    print("데이터가 비어 있어 keys()를 호출할 수 없습니다.")

데이터가 비어 있어 keys()를 호출할 수 없습니다.


In [69]:
data_dict["response"]["body"]["items"].keys()

AttributeError: 'NoneType' object has no attribute 'keys'

In [65]:
print(data_dict["response"]["body"]["items"].keys())

AttributeError: 'NoneType' object has no attribute 'keys'

In [64]:
items = data_dict["response"]["body"]["items"]["item"]
print(len(items))
print(items[0])

TypeError: 'NoneType' object is not subscriptable

In [14]:
import pandas as pd

df = pd.DataFrame(items)

print(df.head(100))

  buildYear contractTerm contractType dealDay dealMonth dealYear deposit  \
0      2001         None         None      21        12     2015   1,000   
1      2003         None         None       9        12     2015   3,000   
2      2013         None         None      12        12     2015   1,000   
3      2004         None         None      15        12     2015   1,000   
4      2013  16.01~17.01           신규      25        12     2015  10,500   
5      2016         None         None      29        12     2015   1,000   
6      2016         None         None      12        12     2015  12,000   
7      2016         None         None      26        12     2015  12,000   
8      2016         None         None      30        12     2015  12,000   
9      2016         None         None      23        12     2015  12,000   

  excluUseAr floor   jibun monthlyRent          offiNm preDeposit  \
0      26.52    11     206          65         대보마로니에텔       None   
1      24.03     7      9

In [ ]:
column_mapping = {
    "sggCd": "행정코드",  # 병합을 위한 핵심 키(Key)
    "offiNm": "오피스텔명",
    "buildYear": "건축년도",
    "floor": "층",
    "excluUseAr": "전용면적",
    "deposit": "보증금",
    "monthlyRent": "월세",
    "dealYear": "계약년도",
    "dealMonth": "계약월",
    "dealDay": "계약일",
    "sggNm": "시군구",
    "umdNm": "법정동",
    "jibun": "지번",
}

df.rename(columns=column_mapping, inplace=True)

print(df.columns)
display(df.head(100))

Index(['건축년도', '계약기간', '계약구분', '계약일', '계약월', '계약년도', '보증금', '전용면적', '층', '지번',
       '월세', '오피스텔명', 'preDeposit', 'preMonthlyRent', '행정코드', '시군구', '법정동',
       'useRRRight', '단위면적당_보증금', '시군구코드'],
      dtype='object')


,건축년도,계약기간,계약구분,계약일,계약월,계약년도,보증금,전용면적,층,지번,월세,오피스텔명,preDeposit,preMonthlyRent,행정코드,시군구,법정동,useRRRight,단위면적당_보증금,시군구코드
0,2001,None,None,21,12,2015,1000,26.52,11,206,65,대보마로니에텔,None,None,11110,종로구,연건동,None,37.707391,종로구
1,2003,None,None,9,12,2015,3000,24.03,7,95,45,코윈 팔레스뷰 오피스텔,None,None,11110,종로구,원남동,None,124.843945,종로구
2,2013,None,None,12,12,2015,1000,25.27,10,1421,55,버넷하임1,None,None,11110,종로구,숭인동,None,39.572616,종로구
3,2004,None,None,15,12,2015,1000,29.41,7,71,80,경희궁의 아침 2단지,None,None,11110,종로구,내수동,None,34.002040,종로구
4,2013,16.01~17.01,신규,25,12,2015,10500,17.88,7,1392-1,0,종로 아인스빌,None,None,11110,종로구,숭인동,None,587.248322,종로구
5,2016,None,None,29,12,2015,1000,21.36,13,1396,60,아이비씨 호텔 앤 오피스텔,None,None,11110,종로구,숭인동,None,46.816479,종로구
6,2016,None,None,12,12,2015,12000,21.36,18,1396,0,아이비씨 호텔 앤 오피스텔,None,None,11110,종로구,숭인동,None,561.797753,종로구
7,2016,None,None,26,12,2015,12000,21.59,15,1396,0,아이비씨 호텔 앤 오피스텔,None,None,11110,종로구,숭인동,None,555.812876,종로구
8,2016,None,None,30,12,2015,12000,21.62,8,1396,0,아이비씨 호텔 앤 오피스텔,None,None,11110,종로구,숭인동,None,555.041628,종로구
9,2016,None,None,23,12,2015,12000,21.62,13,1396,0,아이비씨 호텔 앤 오피스텔,None,None,11110,종로구,숭인동,None,555.041628,종로구


In [ ]:
# 한글로 바꾼 뒤의 분석 예시
df["단위면적당_보증금"] = df["보증금"] / df["전용면적"]

In [ ]:
# 1. 타입 일치 (숫자형이면 앞자리 '0'이 사라질 수 있으므로 문자열로 변환)
df["행정코드"] = df["행정코드"].astype(str)
df_migration["행정코드"] = df_migration["행정코드"].astype(str)

# 2. 자릿수 맞추기 (전월세 데이터는 보통 5자리입니다)
# 만약 순이동률 데이터의 코드가 10자리라면 앞의 5자리만 슬라이싱합니다.
# df_migration['행정코드'] = df_migration['행정코드'].str[:5]

# 3. 데이터 병합
merged_df = pd.merge(df, df_migration, on="행정코드", how="inner")

# 4. 병합 후 데이터 확인
print(f"병합 후 데이터 건수: {len(merged_df)}")
display(merged_df[["시군구", "행정코드", "보증금", "순이동률"]].head())

NameError: name 'df_migration' is not defined

데이터 정제

In [ ]:
# 1. 보증금과 월세에서 콤마(,)를 제거하고 숫자로 변환
df["deposit"] = df["deposit"].str.replace(",", "").astype(int)
df["monthlyRent"] = df["monthlyRent"].astype(int)

# 2. 전용면적을 실수(float)로 변환
df["excluUseAr"] = df["excluUseAr"].astype(float)

# 3. 제대로 바뀌었는지 확인 (기초 통계량 보기)
print(df[["deposit", "monthlyRent", "excluUseAr"]].describe())

            deposit  monthlyRent  excluUseAr
count     10.000000    10.000000   10.000000
mean    6550.000000    30.500000   23.066000
std     5479.507277    33.287468    3.283617
min     1000.000000     0.000000   17.880000
25%     1000.000000     0.000000   21.417500
50%     6750.000000    22.500000   21.620000
75%    12000.000000    58.750000   24.960000
max    12000.000000    80.000000   29.410000


In [ ]:
import pandas as pd

# 1. 분석 대상 코드 리스트 정의
target_codes = {
    "11110": "서울(기준)",
    "44200": "충남 아산",
    "46840": "전남 무안",
    "45710": "전북 완주",
    "44800": "충남 홍성",
    "47170": "경북 안동",
}

# 2. 데이터 전처리 및 필터링
# 전월세 데이터(df)와 순이동(df_mig)에서 해당 코드들만 추출
df_rent = nationwide_df.copy()
df_mig_target = df_mig.copy()

# 행정코드 정리 (문자열 5자리)
df_rent["행정코드"] = df_rent["sggCd"].astype(str).str.zfill(5)
df_mig_target["행정코드"] = df_mig_target["행정코드"].astype(str).str.zfill(5)

# 타겟 지역만 남기기
df_rent = df_rent[df_rent["행정코드"].isin(target_codes.keys())]
df_mig_target = df_mig_target[df_mig_target["행정코드"].isin(target_codes.keys())]

# 3. 주거비 증감 분석 (환산보증금 계산)
df_rent["보증금"] = df_rent["보증금"].astype(str).str.replace(",", "").astype(float)
df_rent["면적당_주거비"] = (
    df_rent["보증금"] + (df_rent["월세"].astype(float) * 100)
) / df_rent["전용면적"].astype(float)

# 4. 지역별 요약
summary = df_rent.groupby("행정코드").agg({"면적당_주거비": "mean"}).reset_index()
summary["지역명"] = summary["행정코드"].map(target_codes)

# 5. 순이동률 합치기
summary = pd.merge(
    summary, df_mig_target[["행정코드", "청년순이동률"]], on="행정코드", how="left"
)

# [중요] 서울 데이터가 순이동률에 없을 경우 수동 패치 (-1.1% 가정)
summary.loc[summary["행정코드"] == "11110", "청년순이동률"] = -1.1

# 6. 🌟 서울 대비 주거비 지수 계산
seoul_price = summary.loc[summary["행정코드"] == "11110", "면적당_주거비"].values[0]
summary["주거비_지수(서울=100)"] = (summary["면적당_주거비"] / seoul_price) * 100

print("📊 서울 대비 타겟 지역 분석 결과")
display(
    summary[
        ["지역명", "면적당_주거비", "주거비_지수(서울=100)", "청년순이동률"]
    ].sort_values("주거비_지수(서울=100)")
)

KeyError: "['청년순이동률'] not in index"

In [ ]:
import pandas as pd

# 1. 순이동률 데이터 불러오기
# KOSIS 데이터는 보통 cp949 인코딩을 사용합니다.
df_mig = pd.read_csv("./data/youth_migration.csv", encoding="cp949")

# 2. [핵심] 병합을 위해 '행정코드' 타입을 문자열로 통일하기
# 데이터 수집 시 타입이 다르면(int vs str) 병합이 되지 않으므로 반드시 실행해야 합니다.
df_mig["행정코드"] = df_mig["행정코드"].astype(str)
df["시군구코드"] = df["시군구"].astype(str)  # 앞서 수집한 전월세 데이터의 지역코드

# 3. 데이터 병합 (Left Join)
# 전월세 데이터(df)를 기준으로 순이동률(df_mig)을 붙입니다.
merged_df = pd.merge(df, df_mig, left_on="시군구코드", right_on="행정코드", how="left")

print("✅ 병합 완료!")
display(merged_df.head())

✅ 병합 완료!


,건축년도,계약기간,계약구분,계약일,계약월,계약년도,보증금,전용면적,층,지번,...,법정동,useRRRight,단위면적당_보증금,시군구코드,행정코드_y,지역명,순이동_1년,순이동_2년,순이동_3년,평균_청년순이동률
0,2001,None,None,21,12,2015,1000,26.52,11,206,...,연건동,None,37.707391,종로구,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,None,None,9,12,2015,3000,24.03,7,95,...,원남동,None,124.843945,종로구,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,None,None,12,12,2015,1000,25.27,10,1421,...,숭인동,None,39.572616,종로구,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,None,None,15,12,2015,1000,29.41,7,71,...,내수동,None,34.002040,종로구,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,16.01~17.01,신규,25,12,2015,10500,17.88,7,1392-1,...,숭인동,None,587.248322,종로구,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 1. 분석에 사용할 핵심 컬럼 리스트 정의
# '행정코드', '시군구'는 기본, '순이동률'과 주거비 관련 지표만 남깁니다.
core_columns = ["행정코드", "시군구", "전용면적", "보증금", "월세", "순이동률"]

# 2. 데이터프레임 슬림화
df_clean = merged_df[core_columns].copy()

# 3. 주거비 지표 통합 (전처리)
# 보증금 콤마 제거 및 숫자 변환
df_clean["보증금"] = df_clean["보증금"].str.replace(",", "").astype(float)
df_clean["월세"] = df_clean["월세"].astype(float)

# 면적당 환산주거비 계산 (월세*100 + 보증금) / 면적
df_clean["면적당_주거비"] = (df_clean["보증금"] + (df_clean["월세"] * 100)) / df_clean[
    "전용면적"
]

# 4. 시군구별 평균 데이터로 요약
df_summary = (
    df_clean.groupby(["행정코드", "시군구"])
    .agg({"면적당_주거비": "mean", "순이동률": "mean"})
    .reset_index()
)

KeyError: "['행정코드', '순이동률'] not in index"

In [44]:
# 병합된 데이터프레임의 실제 컬럼명을 모두 출력합니다.
print(merged_df.columns.tolist())

['건축년도', '계약기간', '계약구분', '계약일', '계약월', '계약년도', '보증금', '전용면적', '층', '지번', '월세', '오피스텔명', 'preDeposit', 'preMonthlyRent', '행정코드_x', '시군구', '법정동', 'useRRRight', '단위면적당_보증금', '시군구코드', '행정코드_y', '지역명', '순이동_1년', '순이동_2년', '순이동_3년', '평균_청년순이동률']


In [ ]:
import pandas as pd

# 1. 사용할 컬럼들만 골라내기 (이름표 정확히 일치시킴)
# 행정코드_x를 대표 코드로 사용하고, 순이동률은 '청년순이동률'을 사용합니다.
df_final = merged_df[
    ["행정코드_x", "시군구", "오피스텔명", "전용면적", "보증금", "월세", "청년순이동률"]
].copy()

# 2. 이름 깔끔하게 정리
df_final.rename(
    columns={"행정코드_x": "행정코드", "청년순이동률": "순이동률"}, inplace=True
)

# 3. 데이터 타입 변환 및 계산
df_final["보증금"] = df_final["보증금"].astype(str).str.replace(",", "").astype(float)
df_final["월세"] = df_final["월세"].astype(float)
df_final["전용면적"] = df_final["전용면적"].astype(float)

# 면적당 주거비(환산보증금 기준) = (보증금 + 월세 * 100) / 전용면적
df_final["면적당_주거비"] = (df_final["보증금"] + (df_final["월세"] * 100)) / df_final[
    "전용면적"
]

# 시군구 단위로 요약
df_summary = (
    df_final.groupby(["행정코드", "시군구"])
    .agg({"면적당_주거비": "mean", "순이동률": "mean"})
    .reset_index()
)

KeyError: "['행정코드_x', '청년순이동률'] not in index"

In [ ]:
import pandas as pd

# 1. 원본 보호를 위한 복사
# nationwide_df(전월세)와 df_mig(순이동)가 살아있는지 확인하세요.
analysis_target = nationwide_df.copy()
migration_data = df_mig.copy()

# 2. [서울 데이터 패치] 순이동 데이터에 서울(11110)이 없으므로 강제 주입
# 2015년 서울 종로구 청년 순이동률 통계치(-1.1)를 예시로 넣습니다.
if not migration_data["행정코드"].astype(str).str.startswith("11").any():
    seoul_patch = pd.DataFrame(
        {"행정코드": ["11110"], "시군구": ["서울 종로구"], "청년순이동률": [-1.1]}
    )
    migration_data = pd.concat([migration_data, seoul_patch], ignore_index=True)


# 3. 행정코드 형식 통일 (문자열 5자리)
def to_code5(x):
    try:
        return str(int(float(x)))[:5].zfill(5)
    except:
        return None


analysis_target["행정코드"] = (
    analysis_target["sggCd"].apply(to_code5)
    if "sggCd" in analysis_target.columns
    else analysis_target["행정코드"].apply(to_code5)
)
migration_data["행정코드"] = migration_data["행정코드"].apply(to_code5)

# 4. 데이터 병합 (Inner Join)
# 'on'을 명시하면 행정코드_x 대신 그냥 '행정코드' 하나만 남습니다.
final_merged = pd.merge(analysis_target, migration_data, on="행정코드", how="inner")

# 5. [중요] 컬럼 정리 및 주거비 계산
if not final_merged.empty:
    # 보증금 숫자 변환 (콤마 제거)
    final_merged["보증금"] = (
        final_merged["보증금"].astype(str).str.replace(",", "").astype(float)
    )
    final_merged["월세"] = final_merged["월세"].astype(float)
    final_merged["전용면적"] = final_merged["전용면적"].astype(float)

    # 면적당 주거비 계산
    final_merged["면적당_주거비"] = (
        final_merged["보증금"] + (final_merged["월세"] * 100)
    ) / final_merged["전용면적"]

    # 6. 시군구별 요약
    # df_mig의 순이동률 컬럼명이 '청년순이동률'인지 확인 후 맞춰줍니다.
    mig_col = "청년순이동률" if "청년순이동률" in final_merged.columns else "순이동률"

    summary = (
        final_merged.groupby(
            ["행정코드", "시군구_y" if "시군구_y" in final_merged.columns else "시군구"]
        )
        .agg({"면적당_주거비": "mean", mig_col: "mean"})
        .reset_index()
    )

    # 7. 🌟 서울 기준(Baseline) 상대 지수 생성
    # 서울(11110)을 100점으로 잡고 비교합니다.
    seoul_val = summary[summary["행정코드"] == "11110"]
    if not seoul_val.empty:
        s_price = seoul_val["면적당_주거비"].values[0]
        s_mig = seoul_val[mig_col].values[0]

        summary["주거비_지수(서울=100)"] = (summary["면적당_주거비"] / s_price) * 100
        summary["순이동_지수(서울=100)"] = (summary[mig_col] / s_mig) * 100

        print("✅ 분석 완료! 서울 대비 지표가 생성되었습니다.")
        display(summary.sort_values("순이동_지수(서울=100)", ascending=False))
else:
    print(
        "🚨 여전히 병합된 데이터가 0건입니다. nationwide_df에 서울 외 다른 지역이 있는지 확인하세요."
    )

✅ 분석 완료! 서울 대비 지표가 생성되었습니다.


,행정코드,시군구_y,면적당_주거비,청년순이동률,주거비_지수(서울=100),순이동_지수(서울=100)
0,11110,서울 종로구,430.081322,-1.1,100.0,100.0


In [ ]:
# 1. 서울(코드 11로 시작)의 기준값 추출
seoul_data = df_summary[df_summary["행정코드"].str.startswith("11")]
seoul_price_ref = seoul_data["면적당_주거비"].mean()
seoul_mig_ref = seoul_data["순이동률"].mean()

# 2. 상대 지수 계산
df_summary["주거비_지수(서울=100)"] = (
    df_summary["면적당_주거비"] / seoul_price_ref
) * 100
df_summary["순이동_지수(서울=100)"] = (df_summary["순이동률"] / seoul_mig_ref) * 100

# 3. 결과 확인 (서울 제외 상위 10개 유망 지역)
local_stars = df_summary[~df_summary["행정코드"].str.startswith("11")].sort_values(
    by="순이동_지수(서울=100)", ascending=False
)
display(local_stars.head(10))

,행정코드,시군구,면적당_주거비,순이동률,주거비_지수(서울=100),순이동_지수(서울=100)


In [47]:
local_stars.head(5)

,행정코드,시군구,면적당_주거비,순이동률,주거비_지수(서울=100),순이동_지수(서울=100)


In [ ]:
# 1. 두 데이터의 행정코드를 모두 '문자열'로 바꾸고 앞 5자리만 남깁니다.
merged_df["행정코드_x"] = merged_df["행정코드_x"].astype(str).str[:5]
merged_df["행정코드_y"] = merged_df["행정코드_y"].astype(str).str[:5]

# 2. 제대로 정리가 되었는지 상위 5개만 먼저 확인 (냉철한 검증)
print(f"전월세 코드 샘플: {merged_df['행정코드_x'].iloc[0]}")
print(f"순이동 코드 샘플: {merged_df['행정코드_y'].iloc[0]}")

# 3. 만약 위 출력값이 둘 다 '11110' 처럼 5자리로 똑같다면 다시 병합 시도
# (이미 merge된 상태라면 아래처럼 컬럼을 다시 정리하세요)
df_final = merged_df[merged_df["행정코드_x"] == merged_df["행정코드_y"]].copy()

if df_final.empty:
    print(
        "🚨 여전히 데이터가 0건입니다. 행정코드 값 자체가 서로 다른지 확인이 필요합니다."
    )
else:
    # 4. 이름 정리 및 요약 (이전 로직 동일)
    df_final.rename(
        columns={"행정코드_x": "행정코드", "청년순이동률": "순이동률"}, inplace=True
    )
    # ... 이후 보증금 숫자 변환 및 서울 기준 지수 계산 진행

전월세 코드 샘플: 11110
순이동 코드 샘플: nan
🚨 여전히 데이터가 0건입니다. 행정코드 값 자체가 서로 다른지 확인이 필요합니다.


In [ ]:
# 순이동률 데이터의 상위 5개를 출력해서 '진짜 모습'을 봅니다.
print("--- 순이동률 데이터(df_mig) 원본 확인 ---")
print(df_mig.head())
print("\n--- 행정코드 컬럼의 데이터 타입 확인 ---")
print(df_mig["행정코드"].dtype)


--- 순이동률 데이터(df_mig) 원본 확인 ---
    행정코드  지역명  순이동_1년  순이동_2년   순이동_3년  평균_청년순이동률
0  32010  춘천시   628.0  1173.0   9782.0   1.233333
1  32020  원주시  -193.0  -116.0   4008.0   0.400000
2  32030  강릉시  -939.0 -1882.0 -10278.0  -2.150000
3  32040  동해시  -407.0  -823.0  -4762.0  -2.600000
4  32050  태백시  -355.0  -691.0  -3810.0  -5.800000

--- 행정코드 컬럼의 데이터 타입 확인 ---
object
